In [1]:
# generate PQ

def generate_pq (G, gens):
    pq = set()
    queue = [g for g in gens]
    
    N = len(G)
    
    rs = G.rewriting_system()
    rs.make_confluent()
    
    while len(queue) > 0:
        gpre = queue.pop()
        
        g = rs.reduce(gpre)
        
        if g in pq:
            continue
        
        for x in pq:
            queue.append(x*g*(x^(-1)))
            queue.append(g*x*(g^(-1)))
        
        for n in range(N):
            queue.append(g^n)
        
        pq.add(g)
        
    return pq

In [2]:
def get_gap_group(n, i):
    return PermutationGroup(gap_group = gap.Image(gap.IsomorphismPermGroup(gap.SmallGroup(n, i)))).as_finitely_presented_group()

In [3]:
G = get_gap_group(64, 135)

In [4]:
G

Finitely presented group < a, b, c, d, e, f | a^2, b^2, e^2, f^2, d^-2*f, f*c^-2, d*e*d^-1*e, c*e*c^-1*e, (a*e)^2, c*d^-1*c^-1*d^-1, (b*d^-1)^2, (a*d^-1)^2, b*c*b*c^-1, e*c*a*c^-1*a, d^-1*(b*a)^2 >

In [19]:
G.structure_description()

'(C8 : (C2 x C2)) : C2'

In [5]:
G.gens()

(a, b, c, d, e, f)

In [6]:
pq = generate_pq(G, G.gens()); pq

{1,
 a,
 b,
 c,
 d,
 e,
 f,
 a*d,
 a*e,
 a*f,
 b*d,
 b*f,
 c*e,
 c^-1,
 d^-1,
 e*f,
 a*d*e,
 a*d^-1,
 a*e*f,
 b*d^-1,
 c^-1*e,
 a*d^-1*e}

In [7]:
len(pq)

22

In [8]:
len(G)

64

In [45]:
list(enumerate(G))

[(0, 1),
 (1, a),
 (2, b),
 (3, c),
 (4, c^-1),
 (5, d),
 (6, d^-1),
 (7, e),
 (8, f),
 (9, a*b),
 (10, a*c),
 (11, a*c^-1),
 (12, a*d),
 (13, a*d^-1),
 (14, a*e),
 (15, a*f),
 (16, b*a),
 (17, b*c),
 (18, b*c^-1),
 (19, b*d),
 (20, b*d^-1),
 (21, b*e),
 (22, b*f),
 (23, c*a),
 (24, c*d),
 (25, c*d^-1),
 (26, c*e),
 (27, c^-1*a),
 (28, c^-1*e),
 (29, d*e),
 (30, d^-1*e),
 (31, e*b),
 (32, e*f),
 (33, a*b*c),
 (34, a*b*c^-1),
 (35, a*b*d^-1),
 (36, a*b*e),
 (37, a*b*f),
 (38, a*c*d),
 (39, a*c*d^-1),
 (40, a*d*e),
 (41, a*d^-1*e),
 (42, a*e*b),
 (43, a*e*f),
 (44, b*a*c),
 (45, b*a*c^-1),
 (46, b*a*e),
 (47, b*c*a),
 (48, b*c*d),
 (49, b*c*d^-1),
 (50, b*c*e),
 (51, b*c^-1*a),
 (52, b*c^-1*e),
 (53, b*d*e),
 (54, b*d^-1*e),
 (55, c*a*b),
 (56, c*a*d),
 (57, c*a*d^-1),
 (58, c*d*e),
 (59, c*d^-1*e),
 (60, c^-1*a*b),
 (61, e*b*a),
 (62, a*b*c*a),
 (63, a*b*c^-1*a)]

In [66]:
custom_gen = (list(G)[13], list(G)[20], list(G)[62], list(G)[5], list(G)[9], list(G)[36]); custom_gen

(a*d^-1, b*d^-1, a*b*c*a, d, a*b, a*b*e)

In [67]:
pq1 = generate_pq(G, custom_gen); pq1

{1,
 a,
 b,
 d,
 f,
 a*b,
 a*d,
 a*e,
 a*f,
 b*c,
 b*d,
 b*f,
 d^-1,
 b*a,
 a*b*e,
 a*b*f,
 a*d*e,
 a*d^-1,
 a*e*f,
 b*c^-1,
 b*d^-1,
 b*a*e,
 a*b*d^-1,
 a*e*b,
 a*d^-1*e,
 a*b*c^-1*a,
 e*b*a,
 a*b*c*a}

In [68]:
len(pq1)

28

In [9]:

enumed = list(enumerate(pq))
#gens = list(G.gens())

rs = G.rewriting_system()
rs.make_confluent()

groupIndex = dict([(rs.reduce(g), i) for i, g in enumed])

def indexOf(g):
    g1 = rs.reduce(g)
    return groupIndex[g1]
    #for i, g1 in enumed:
        #if g == g1:
            #return i

def forOneG(t):
    (i, g) = t
    grelations = []
    
    indexInv = indexOf(g^(-1))
    
    for n in range(len(G)):
        gn = g^n
        if gn == g and n > 1:
            break
        indexN = indexOf(gn)
        grelations.append([indexInv + 1]*n + [indexN + 1])

    for i1, g1 in enumed:
        #i1 = indexOf(g1)
        indexInv1 = indexOf(g1^(-1))
        indexInv2 = indexOf((g1*g*g1^(-1))^(-1)) 
        grelations.append([i1 + 1, i + 1, indexInv1 + 1, indexInv2 + 1])

    return grelations

relations = []

#for i, g in enumed:
    #rel = forOneG(i, g)
    #relations.extend(rel)

from multiprocessing import Pool

pool = Pool(processes = 7)
result = pool.map(forOneG, enumed)
relations = [rel for rels in result for rel in rels]
pool.close()
pool.join()

In [10]:
FreeG = FreeGroup(len(pq))

In [11]:
FG = FreeG / relations

In [12]:
FGS = FG.simplified()
FGS

Finitely presented group < x1, x2, x3, x7, x16 | x2^2, x3^2, x16^2, x7*x1^-2*x7, x1*x7^-1*x1*x7, x16*x7^-1*x16*x7, (x3*x2)^2, (x1^-1*x3)^2, x1^-1*x2*x1*x2, x2*x7^-1*x2*x7, (x16*x1^-1)^2, x2*x1^-1*x16*x1*x2*x16, (x3*x16*x3*x7)^2, (x16*x3*x16*x2)^2, (x7*x3)^4, x3*x16*x2*(x3*x16)^3*x2 >

In [13]:
FGS.structure_description()

'C2 x C2 x (((C2 x Q8) : C2) : C2)'

In [14]:
FGS.gens()

(x1, x2, x3, x7, x16)

In [16]:
FGS.order()

256

In [15]:
FGS.is_isomorphic(G)

False